1.	Для чего и в каких случаях полезны различные варианты усреднения для метрик качества классификации: micro, macro, weighted?

Макро — вычисление метрики для каждого класса и получение невзвешенного среднего; вычислит метрику независимо для каждого класса, а затем возьмет среднее (следовательно, будет обрабатывать все классы одинаково).
Микро — расчет метрики на глобальном уровне путем подсчета итоговых истинных положительных результатов, ложноотрицательных и ложноположительных результатов (независимо от классов); микро-усреднения суммируют отдельные истинные положительные, ложные положительные и отрицательные значения системы для разных наборов и применяете их для получения статистики
При микро-усреднении сначала характеристики усредняются по всем классам, а затем вычисляется итоговая двухклассовая метрика например, точность, полно- та или F-мера. При макро-усреднении сначала вычисляется итоговая метрика для каждого класса, а затем результаты усредняются по всем классам.
Взвешенный — вычисление метрик для каждого класса и получение взвешенного среднего по числу выборок на каждый класс.


2.	В чём разница между моделями xgboost, lightgbm и catboost или какие их основные особенности?

lightGBM предлагает градиентную одностороннюю выборку (GOSS), которая выбирает разбиение, используя все экземпляры с большими градиентами (т.е. с большой ошибкой) и случайную выборку экземпляров с малыми градиентами. Чтобы сохранить одинаковое распределение данных при вычислении информационного выигрыша, GOSS вводит постоянный множитель для экземпляров данных с малыми градиентами. Таким образом, GOSS достигает хорошего баланса между увеличением скорости за счет уменьшения количества экземпляров данных и сохранением точности для выученных деревьев решений. 
LightGBM использует рост дерева по листьям (по принципу "лучший-первый"). Он выбирает для роста тот лист, который минимизирует потери, что позволяет вырастить несбалансированное дерево. Поскольку дерево растет не по уровням, а по листьям, при малых данных может произойти перебор. В таких случаях важно контролировать глубину дерева.
В LightGBM недостающие значения будут распределяться в ту сторону, которая уменьшает потери в каждом разбиении.
LightGBM разделяет категориальные признаки, разбивая их категории на 2 подмножества. Основная идея заключается в том, чтобы сортировать категории в соответствии с целью обучения при каждом разбиении. 

Catboost предлагает новую технику под названием Minimal Variance Sampling (MVS), которая представляет собой взвешенную выборку версии Stochastic Gradient Boosting. В этой технике взвешенная выборка происходит на уровне дерева, а не на уровне разбиения. Наблюдения для каждого дерева бустинга отбираются таким образом, чтобы максимизировать точность оценки разбиения.
Catboost строит сбалансированное дерево. На каждом уровне такого дерева выбирается пара признак-сплит, которая приносит наименьшие потери (согласно штрафной функции) и используется для всех узлов уровня. Можно изменить его правила с помощью параметра grow-policy.
Catboost имеет два режима обработки отсутствующих значений: "Min" и "Max". В режиме "Min" отсутствующие значения обрабатываются как минимальное значение для признака (им присваивается значение, которое меньше всех существующих значений). Таким образом, гарантируется, что при выборе разбиения будет учитываться разбиение, которое отделяет отсутствующие значения от всех остальных значений. "Max" работает точно так же, как и "Min", только с максимальными значениями.
Catboost имеет два метода: Первый - "PredictionValuesChange". Для каждого признака PredictionValuesChange показывает, насколько в среднем изменяется предсказание при изменении значения признака. Признак будет иметь большую важность, если изменение значения признака вызывает большое изменение в прогнозируемом значении. Это метод расчета важности признака по умолчанию для неранжированных метрик. Второй метод - "LossFunctionChange". Этот тип важности признака может использоваться для любой модели, но особенно полезен для моделей ранжирования. Для каждого признака значение представляет собой разницу между значением потерь модели с этим признаком и без него. Поскольку переобучение модели без одного из признаков требует больших вычислительных затрат, эта модель строится приблизительно, используя исходную модель с этим признаком, удаленным из всех деревьев в ансамбле. Для расчета важности этого признака требуется набор данных.
Catboost использует комбинацию одноточечного кодирования и расширенного среднего кодирования. Для признаков с малым количеством категорий используется одноточечное кодирование. Максимальное количество категорий для одноточечного кодирования можно контролировать с помощью параметра one_hot_max_size. Для остальных категориальных столбцов CatBoost использует эффективный метод кодирования, который похож на среднее кодирование, но с дополнительным механизмом, направленным на снижение переподгонки. Использование категориального кодирования CatBoost имеет обратную сторону - более медленная модель. 

XGboost не использует никаких методов взвешенной выборки, что делает его процесс разбиения более медленным по сравнению с GOSS и MVS.
XGboost разделяет до заданного гиперпараметра max_depth, а затем начинает обрезать дерево в обратном направлении и удаляет части, за пределами которых нет положительного выигрыша. Такой подход используется, поскольку иногда за расщеплением без уменьшения потерь может следовать расщепление с уменьшением потерь. XGBoost также может выполнять рост дерева по листьям (как LightGBM).
XGBoost недостающие значения будут распределяться в ту сторону, которая уменьшает потери в каждом разбиении.
XGBoost не имеет встроенного метода для категориальных признаков. Кодирование (one-hot, целевое кодирование и т.д.) должно выполняться пользователем.

